In [8]:
import torch

In [9]:
import pandas as pd

In [10]:
from EnhancedGATModel import EnhancedGATModel
from GraphBuilder import GraphBuilder
from GraphTrainer import GraphTrainer

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
transactions = pd.read_csv('/AiDev/large_models/clean_transactions.csv')

In [59]:
def GenerateTransactionSet(transactions, legit_multiplier=2, valid_ratio=0.2):
    fraud_mask = transactions['is_fraud']=='Yes'
    legit_mask = ~fraud_mask

    fraud_set = transactions[fraud_mask]    
    legit_set = transactions[legit_mask].sample(len(fraud_set)*legit_multiplier)

    valid_fraud = fraud_set.sample(frac=valid_ratio)
    train_fraud = fraud_set.drop(valid_fraud.index)
    valid_legit = legit_set.sample(frac=valid_ratio)
    train_legit = legit_set.drop(valid_legit.index)

    valid_set = pd.concat([valid_fraud, valid_legit]).sample(frac=1).reset_index()
    valid_set = valid_set.rename(columns={'index':'original_index'})
    train_set = pd.concat([train_fraud, train_legit]).sample(frac=1).reset_index()
    train_set = train_set.rename(columns={'index':'original_index'})
    
    return valid_set[['original_index','user_id','is_fraud']], train_set[['original_index','user_id','is_fraud']]

In [60]:
validation_set, training_set = GenerateTransactionSet(transactions)

In [62]:
model = EnhancedGATModel(hidden_channels=24, out_channels=2)
graph_builder = GraphBuilder()
trainer = GraphTrainer(device)

In [ ]:
trainer.train